In [4]:
import time
import talib
import numpy as np
import pandas as pd
import requests
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException
from apscheduler.schedulers.blocking import BlockingScheduler

# ✅ Enter Your Binance Testnet API Key and Secret Here
API_KEY = "l1axguRUuSx0aXTuwprNcFNHNM7J5HeKfN0OKp85ABAQmH0oWCNci02MQfVB97aY"
API_SECRET = "EicJhiWluIhggMdByykWpAMklPKCLr3R47cj5nggnTZvBfvVVQzta8va60ekD6y7"

# ✅ Initialize Binance Testnet Client
try:
    client = Client(API_KEY, API_SECRET, testnet=True)  # Enable Testnet
    client.ping()  # Ensure API is reachable
    print("✅ Binance Testnet Connection Successful!")
except BinanceAPIException as e:
    print(f"❌ Binance API Error: {e}")
    exit()
except requests.exceptions.RequestException as e:
    print(f"❌ Network Error: {e}")
    exit()

# ✅ Function to Safely Call Binance API with Retry Mechanism
def safe_binance_call(func, *args, **kwargs):
    max_retries = 5
    retry_delay = 5

    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except BinanceAPIException as e:
            if "502 Bad Gateway" in str(e):
                print(f"⚠️ Binance 502 Error (Attempt {attempt+1}/{max_retries}) - Retrying in {retry_delay}s...")
                time.sleep(retry_delay)
            else:
                print(f"❌ Binance API Exception: {e}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"❌ Network Error: {e}")
            return None
    print("❌ Binance API Failed After Multiple Attempts!")
    return None

# ✅ Function to Fetch Historical Data
def get_historical_data(symbol, interval, lookback):
    klines = safe_binance_call(client.get_klines, symbol=symbol, interval=interval, limit=lookback)
    if not klines:
        return None
    
    df = pd.DataFrame(klines, columns=['time', 'open', 'high', 'low', 'close', 'volume', 
                                       'close_time', 'quote_asset_volume', 'num_trades', 
                                       'taker_buy_base_vol', 'taker_buy_quote_vol', 'ignore'])
    
    df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].astype(float)
    return df

# ✅ Function to Identify Candlestick Patterns
def identify_patterns(df):
    patterns = {
        'Hammer': talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close']),
        'Engulfing': talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']),
        'Morning Star': talib.CDLMORNINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Evening Star': talib.CDLEVENINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Shooting Star': talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close']),
        'Doji': talib.CDLDOJI(df['open'], df['high'], df['low'], df['close']),
        'Three White Soldiers': talib.CDL3WHITESOLDIERS(df['open'], df['high'], df['low'], df['close']),
        'Three Black Crows': talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close']),
        'Bullish Harami': talib.CDLHARAMI(df['open'], df['high'], df['low'], df['close']),
        'Bearish Harami': talib.CDLHARAMICROSS(df['open'], df['high'], df['low'], df['close'])
    }
    
    signals = [key for key, val in patterns.items() if val.iloc[-1] != 0]
    return signals

# ✅ Function to Generate Trading Signals
def generate_signal(symbol):
    df = get_historical_data(symbol, Client.KLINE_INTERVAL_15MINUTE, 50)
    if df is None:
        return None, None

    patterns = identify_patterns(df)

    if patterns:
        last_close = df['close'].iloc[-1]
        
        if any(p in patterns for p in ['Hammer', 'Morning Star', 'Engulfing', 'Three White Soldiers', 'Bullish Harami']):
            return "LONG", last_close
        elif any(p in patterns for p in ['Shooting Star', 'Evening Star', 'Three Black Crows', 'Bearish Harami']):
            return "SHORT", last_close

    return None, None

# ✅ Function to Execute Trades on Testnet
def execute_trade(symbol):
    signal, entry_price = generate_signal(symbol)
    if signal:
        print(f"\n📊 {symbol} - {signal} Signal Detected at {entry_price}")
        
        stop_loss = entry_price * (0.97 if signal == "LONG" else 1.03)
        tp1 = entry_price * (1.02 if signal == "LONG" else 0.98)
        tp2 = entry_price * (1.04 if signal == "LONG" else 0.96)
        tp3 = entry_price * (1.06 if signal == "LONG" else 0.94)
        
        print(f"Entry: {entry_price}, Stop Loss: {stop_loss}, TP1: {tp1}, TP2: {tp2}, TP3: {tp3}")
        
        order_type = ORDER_TYPE_MARKET
        side = SIDE_BUY if signal == "LONG" else SIDE_SELL
        
        try:
            order = safe_binance_call(client.create_order,
                symbol=symbol,
                side=side,
                type=order_type,
                quantity=0.01  # Modify lot size for test
            )
            if order:
                print(f"✅ Trade Executed: {order}")
        except Exception as e:
            print(f"⚠️ Trade Execution Error: {e}")

# ✅ Function to Check Placed Trades
def check_trades(symbol):
    orders = safe_binance_call(client.get_all_orders, symbol=symbol)
    if orders:
        print("\n📜 Placed Trades:")
        for order in orders[-5:]:  # Show last 5 trades
            print(f"🔹 Order {order['orderId']} - {order['side']} {order['origQty']} at {order['price']} (Status: {order['status']})")

# ✅ Function to Scan and Trade the Top 100 USDT Pairs
def trading_bot():
    usdt_pairs = [s['symbol'] for s in safe_binance_call(client.get_exchange_info)['symbols'] if s['symbol'].endswith("USDT")]
    
    if not usdt_pairs:
        print("❌ No USDT pairs found!")
        return
    
    for symbol in usdt_pairs[:100]:  
        execute_trade(symbol)
        check_trades(symbol)
        time.sleep(2)

# ✅ Scheduler to Run the Bot Every 5 Minutes
scheduler = BlockingScheduler()
scheduler.add_job(trading_bot, 'interval', minutes=5)
print("🚀 Trading bot started... Press Ctrl+C to stop.")
scheduler.start()

✅ Binance Testnet Connection Successful!
🚀 Trading bot started... Press Ctrl+C to stop.

📊 BTCUSDT - LONG Signal Detected at 90348.36
Entry: 90348.36, Stop Loss: 87637.9092, TP1: 92155.3272, TP2: 93962.2944, TP3: 95769.26160000001
✅ Trade Executed: {'symbol': 'BTCUSDT', 'orderId': 454138, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJ76a2c21828ea952b9fc9ba', 'transactTime': 1741268343970, 'price': '0.00000000', 'origQty': '0.01000000', 'executedQty': '0.01000000', 'origQuoteOrderQty': '0.00000000', 'cummulativeQuoteQty': '903.47990000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1741268343970, 'fills': [{'price': '90347.99000000', 'qty': '0.01000000', 'commission': '0.00000000', 'commissionAsset': 'BTC', 'tradeId': 104345}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}

📜 Placed Trades:
🔹 Order 454138 - BUY 0.01000000 at 0.00000000 (Status: FILLED)

📊 ETHUSDT - LONG Signal Detected at 2264.33
Entry: 2264.33, Stop Loss: 2196.4001, TP1: 2309.

Execution of job "trading_bot (trigger: interval[0:05:00], next run at: 2025-03-06 19:19:02 IST)" skipped: maximum number of running instances reached (1)



📜 Placed Trades:
🔹 Order 454138 - BUY 0.01000000 at 0.00000000 (Status: FILLED)
🔹 Order 457107 - BUY 0.01000000 at 0.00000000 (Status: FILLED)

📊 ETHUSDT - LONG Signal Detected at 2255.0
Entry: 2255.0, Stop Loss: 2187.35, TP1: 2300.1, TP2: 2345.2000000000003, TP3: 2390.3
✅ Trade Executed: {'symbol': 'ETHUSDT', 'orderId': 400610, 'orderListId': -1, 'clientOrderId': 'x-HNA2TXFJc07835411376ac7adbd2e', 'transactTime': 1741269245948, 'price': '0.00000000', 'origQty': '0.01000000', 'executedQty': '0.01000000', 'origQuoteOrderQty': '0.00000000', 'cummulativeQuoteQty': '22.54980000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1741269245948, 'fills': [{'price': '2254.98000000', 'qty': '0.01000000', 'commission': '0.00000000', 'commissionAsset': 'ETH', 'tradeId': 108176}], 'selfTradePreventionMode': 'EXPIRE_MAKER'}

📜 Placed Trades:
🔹 Order 397131 - BUY 0.01000000 at 0.00000000 (Status: FILLED)
🔹 Order 399235 - BUY 0.01000000 at 0.00000000 (Status:

Execution of job "trading_bot (trigger: interval[0:05:00], next run at: 2025-03-06 19:29:02 IST)" skipped: maximum number of running instances reached (1)



📜 Placed Trades:
🔹 Order 454138 - BUY 0.01000000 at 0.00000000 (Status: FILLED)
🔹 Order 457107 - BUY 0.01000000 at 0.00000000 (Status: FILLED)

📜 Placed Trades:
🔹 Order 397131 - BUY 0.01000000 at 0.00000000 (Status: FILLED)
🔹 Order 399235 - BUY 0.01000000 at 0.00000000 (Status: FILLED)
🔹 Order 400610 - BUY 0.01000000 at 0.00000000 (Status: FILLED)

📜 Placed Trades:
🔹 Order 183127 - BUY 0.01000000 at 0.00000000 (Status: FILLED)

📊 LTCUSDT - LONG Signal Detected at 105.89
Entry: 105.89, Stop Loss: 102.7133, TP1: 108.0078, TP2: 110.1256, TP3: 112.24340000000001
❌ Binance API Exception: APIError(code=-1013): Filter failure: NOTIONAL

📊 QTUMUSDT - LONG Signal Detected at 2.345
Entry: 2.345, Stop Loss: 2.2746500000000003, TP1: 2.3919, TP2: 2.4388, TP3: 2.4857000000000005
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 ADAUSDT - LONG Signal Detected at 0.9332
Entry: 0.9332, Stop Loss: 0.905204, TP1: 0.951864, TP2: 0.9705280000000001, TP3: 0.9891920000000001
❌ Binan

KeyboardInterrupt: 


📊 WANUSDT - LONG Signal Detected at 0.1214
Entry: 0.1214, Stop Loss: 0.11775799999999999, TP1: 0.123828, TP2: 0.126256, TP3: 0.128684
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 BANDUSDT - LONG Signal Detected at 0.898
Entry: 0.898, Stop Loss: 0.87106, TP1: 0.91596, TP2: 0.9339200000000001, TP3: 0.9518800000000001
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 RVNUSDT - LONG Signal Detected at 0.01305
Entry: 0.01305, Stop Loss: 0.0126585, TP1: 0.013311000000000002, TP2: 0.013572, TP3: 0.013833000000000002
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 NKNUSDT - LONG Signal Detected at 0.052
Entry: 0.052, Stop Loss: 0.05044, TP1: 0.05304, TP2: 0.054079999999999996, TP3: 0.05512
❌ Binance API Exception: APIError(code=-1013): Filter failure: LOT_SIZE

📊 TROYUSDT - LONG Signal Detected at 0.00132
Entry: 0.00132, Stop Loss: 0.0012804, TP1: 0.0013464, TP2: 0.0013728, TP3: 0.0013992
❌ Binance API Exception